In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import requests, json

np.random.seed(0)

In [2]:
cd data/

D:\Documents\Tugas\Anaconda\Skripsi_Recommender\data_50u_35i\data


In [9]:
names = ['user_id', 'item_id', 'rating']
df = pd.read_csv('reviews_50u_35i_ready.csv', sep=',', names=names)

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [4]:
print(str(n_users) + ' users')
print(str(n_items) + ' items')
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

8625 users
25877 items
Sparsity: 0.41%


In [5]:
from ExplicitMF import ExplicitMF

In [6]:
best_sgd_model = ExplicitMF(ratings, n_factors=20, learning='sgd', \
                            item_fact_reg=0.1, user_fact_reg=0.1, \
                            user_bias_reg=0.1, item_bias_reg=0.1)
best_sgd_model.train(200, learning_rate=0.001)

In [7]:
from IPython.display import HTML

def display_top_k_books(similarity, idx_to_book, book_idx, k=10):
    book_indices = np.argsort(similarity[book_idx,:])[::-1]
    book_idx_list = ''
    k_ctr = 0
    # Start i at 1 to not grab the input book
    i = 0
    while k_ctr < 10:
        book = idx_to_book[book_indices[i]]
        if book != '':
            book_idx_list += "<h4>- %s<h4/>"\
                      % book
            k_ctr += 1
        i += 1
    display(HTML(book_idx_list))

def compare_recs(sgd_similarity,\
                 idx_to_book, book_idx, k=10):

    # Display SGD Recs
    display(HTML('<font size=5>'+'SGD Recs'+'</font>'))
    display_top_k_books(sgd_similarity,\
                          idx_to_book, book_idx)

In [8]:
sgd_sim = best_sgd_model.cosine_similarity()

In [11]:
import io
# Load in book data
idx_to_book = {}

with io.open('reviews_50u_35i_info_items.csv', mode='r', encoding='utf-8-sig') as f:
    for line in f.readlines():
        info = line.split(',')        
        if len(info[1]) <= 10:
            for asin_char in range(len(info[1]), 11):
                info[1] = '0' + info[1]
        
        idx_to_book[int(info[0])-1] = info[0] + ' : ' + info[1]    

In [12]:
idx = 20000
compare_recs(sgd_sim, idx_to_book, idx)

In [3]:
import pickle

In [7]:
with open("python_best_sgd_model.pkl", "wb") as file_handler: 
    pickle.dump(best_sgd_model, file_handler, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open("python_best_sgd_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

In [18]:
users = ratings.shape[0]
books = ratings.shape[1]
books_mean = np.zeros((1, books))
books_norm = np.zeros((users, books))

R = np.zeros((users,books))
for us in range(users):
    for mo in range(books):
        if ratings[us,mo] >= 1.0:
            R[us,mo] = 1
        else:
            R[us,mo] = 0

for i in range(books):
    idx = np.where(R[:,i] == 1)[0]
    books_mean[0][i] = ratings[idx,i].mean()
    books_norm[idx,i] = ratings[idx,i] - books_mean[0][i]

In [21]:
best_sgd_model2 = ExplicitMF(books_norm, n_factors=20, learning='sgd', \
                            item_fact_reg=0.1, user_fact_reg=0.1, \
                            user_bias_reg=0.1, item_bias_reg=0.1)
best_sgd_model2.train(200, learning_rate=0.001)

In [22]:
with open("python_best_sgd_model2.pkl", "wb") as file_handler2:
    pickle.dump(best_sgd_model2, file_handler2, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open("python_best_sgd_model2.pkl", "rb") as file_handler2:
    loaded_pickle2 = pickle.load(file_handler2)
    
loaded_pickle2

In [19]:
my_ratings = np.zeros((loaded_pickle2.item_vecs.shape[0], 1))

#1st set:
# my_ratings[0] = 5
# my_ratings[1] = 5
# my_ratings[2] = 5
# my_ratings[3] = 5
# my_ratings[4] = 5
# my_ratings[5] = 5
# my_ratings[6] = 5
# my_ratings[7] = 5
# my_ratings[8] = 5
# my_ratings[9] = 5
# my_ratings[10] = 5
# my_ratings[11] = 5
# my_ratings[12] = 4
# my_ratings[13] = 4
# my_ratings[14] = 4
# my_ratings[15] = 4
# my_ratings[16] = 4
# my_ratings[17] = 1
# my_ratings[18] = 1
# my_ratings[19] = 2
# my_ratings[20] = 1


#2nd set:
my_ratings[20000] = 5
my_ratings[20001] = 5
my_ratings[20002] = 5
my_ratings[20003] = 5
my_ratings[20004] = 5
my_ratings[20005] = 5
my_ratings[20006] = 5
my_ratings[20007] = 5
my_ratings[20008] = 5
my_ratings[20009] = 5
my_ratings[20010] = 5
my_ratings[20011] = 5
my_ratings[20012] = 4
my_ratings[20013] = 4
my_ratings[20014] = 4
my_ratings[20015] = 4
my_ratings[20016] = 4
my_ratings[20017] = 1
my_ratings[20018] = 1
my_ratings[20019] = 2
my_ratings[20020] = 1

In [20]:
new_user_rec = my_ratings.T.dot(loaded_pickle2.item_vecs)\
                    .dot(loaded_pickle2.item_vecs.T)
    
new_user_rec

array([[ 0.04369544,  0.24075334,  0.24938365, ...,  0.29949133,
        -0.11070246, -0.06141413]])

In [21]:
sorted_preds = np.sort(new_user_rec[0,:], axis=0)[::-1]
sorted_preds[:30]

array([1.92374481, 1.2438154 , 1.22794331, 1.17376114, 1.17282201,
       1.15492362, 1.06399588, 1.00719291, 0.97520898, 0.95572669,
       0.93732068, 0.93600036, 0.92658544, 0.91467604, 0.90574973,
       0.89277848, 0.88935366, 0.87062571, 0.86831889, 0.86595545,
       0.86010037, 0.85162098, 0.84672255, 0.84598773, 0.84577647,
       0.84554183, 0.84222805, 0.83312344, 0.82749526, 0.82089329])

In [22]:
import io
# Load in book data
idx_to_book = {}

with io.open('reviews_50u_35i_info_items.csv', mode='r', encoding='utf-8-sig') as f:
    for line in f.readlines():
        info = line.split(',')        
        if len(info[1]) <= 10:
            for asin_char in range(len(info[1]), 11):
                info[1] = '0' + info[1]
        
        idx_to_book[int(info[0])-1] = info[0] + ' : ' + info[1]    

In [23]:
from IPython.display import HTML

In [24]:
def display_only(new_user_rec, idx_to_book):
    my_idx = np.argsort(new_user_rec[0,:], axis=0)[::-1]
    book_idx_list = ''
    k_ctr = 0
    # Start i at 1 to not grab the input book
    i = 0
    while k_ctr < 10:
        book = idx_to_book[my_idx[i]]
        if book != '':
            book_idx_list += "<h4>- %s<h4/>"\
                      % book
            k_ctr += 1
        i += 1
    display(HTML(book_idx_list))

print("Top 10 book recommendation")
display_only(new_user_rec, idx_to_book)

Top 10 book recommendation
